# feature Selection

### 递归特征消除

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

def rfe_feature_selection(df, target, n_features_to_select=40):
    """
    使用递归特征消除（RFE）对数据进行降维。
    
    参数:
    df (DataFrame): 包含特征的 DataFrame。
    target (Series): 目标变量（y）。
    n_features_to_select (int): 要选择的特征数量。

    返回:
    selected_features_df (DataFrame): 包含被选择特征的 DataFrame。
    """
    # 初始化线性回归模型
    model = LinearRegression()

    # 初始化RFE
    rfe = RFE(estimator=model, n_features_to_select=n_features_to_select)

    # 拟合RFE模型
    rfe.fit(df, target)

    # 筛选出被选择的特征
    selected_features = df.columns[rfe.support_]

    # 返回包含被选择特征的DataFrame
    selected_features_df = df[selected_features]

    return selected_features_df
selected_features_df = rfe_feature_selection(df_short_x_scaled, df_short_y_scaled, n_features_to_select=40)
# list(selected_features_df)


### 互信息法

In [ ]:
from sklearn.feature_selection import mutual_info_regression
import pandas as pd

def mutual_info_feature_selection(df, target, n_features_to_select=40):
    """
    使用互信息法选择最重要的特征（针对回归任务）。
    
    参数:
    df (DataFrame): 包含特征的 DataFrame。
    target (Series): 目标变量（y）。
    n_features_to_select (int): 要选择的特征数量。

    返回:
    selected_features_df (DataFrame): 包含被选择特征的 DataFrame。
    """
    # 计算互信息分数
    mi_scores = mutual_info_regression(df, target)

    # 将分数转换为Series并排序
    mi_scores = pd.Series(mi_scores, index=df.columns)
    mi_scores_sorted = mi_scores.sort_values(ascending=False)

    # 选择前n_features_to_select个特征
    selected_features = mi_scores_sorted.head(n_features_to_select).index

    # 返回包含被选择特征的DataFrame
    selected_features_df = df[selected_features]

    return selected_features_df
selected_features_df = mutual_info_feature_selection(df_short_x_scaled, df_short_y_scaled, n_features_to_select=40)
# list(selected_features_df)


### 根据相关系数剔除

In [ ]:
import pandas as pd

def analyze_high_correlation(df, threshold=0.9):
    """
    分析DataFrame中每个变量与其他变量的相关系数，并找出相关系数高于阈值的变量数量。

    参数:
    df : pandas.DataFrame
        要分析的DataFrame。
    threshold : float
        相关系数的阈值。

    返回:
    pandas.DataFrame
        包含每个变量和其相关系数高于阈值的变量数量的DataFrame。
    """
    # 计算相关系数矩阵并取绝对值
    corr_matrix = df.corr().abs()

    # 初始化一个Series来保存每个变量的相关系数大于阈值的变量数量
    count_high_corr = pd.Series(0, index=df.columns)

    # 遍历相关系数矩阵，统计每个变量的相关系数大于阈值的数量
    for col in df.columns:
        # 对角线上的值永远是1，排除自身
        count_high_corr[col] = (corr_matrix[col] > threshold).sum() - 1

    # 创建一个DataFrame来保存统计结果
    high_corr_df = pd.DataFrame({
        'Variable': count_high_corr.index,
        'Count_High_Corr': count_high_corr
    }).sort_values('Count_High_Corr', ascending=False)

    # 返回结果DataFrame
    return high_corr_df 




In [ ]:
def drop_highly_correlated_features(df, target_feature, threshold=0.9):
    """
    剔除与目标特征相关系数高于阈值的变量。

    参数:
    df : pandas.DataFrame
        要处理的DataFrame。
    target_feature : str
        目标特征的名称，将基于这个特征的相关系数来剔除其他变量。
    threshold : float
        相关系数的阈值，超过此阈值的变量将被剔除。
    """
    # 计算与目标特征的相关系数并剔除目标特征自身
    corr_with_target = df.corrwith(df[target_feature]).drop(target_feature)

    # 找到相关系数高于阈值的变量
    high_corr_vars = corr_with_target[corr_with_target > threshold].index.tolist()

    # 剔除这些变量
    df = df.drop(columns=high_corr_vars)

    # 显示被剔除的变量名单
    print(f"与 '{target_feature}' 相关系数高于 {threshold} 的被剔除变量名单：", high_corr_vars)
    
    return df